In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

EPOCHS = 50
BATCH_SIZE = 64

train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                         train=True,
                         download=True,
                         transform=transforms.Compose([
                         transforms.RandomHorizontalFlip(),
                         transforms.ToTensor(),
                         transforms.Normalize((0.1307,), (0.3081,))
                        ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                         train=False,
                         transform=transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.1307,),(0.3081,))
                        ])),
    batch_size=BATCH_SIZE, shuffle=True)

class Net(nn.Module):
  def __init__(self, dropout_p=0.2):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(784,256)
    self.fc2 = nn.Linear(256,128)
    self.fc3 = nn.Linear(128,10)
    self.dropout_p = dropout_p

  def forward(self, x):
    x = x.view(-1, 784)
    x = F.relu(self.fc1(x))

    #드롭아웃 추가
    x = F.dropout(x, training=self.training,
                  p=self.dropout_p)
    x = F.relu(self.fc2(x))

    #드롭아웃 추가
    x = F.dropout(x, training=self.training,
                  p=self.dropout_p)
    x = self.fc3(x)
    return x

model = Net(dropout_p = 0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr = 0.01)

def train(model, train_loader, optimizer):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(DEVICE), target.to(DEVICE)
    optimizer.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(DEVICE), target.to(DEVICE)
      output = model(data)
      test_loss += F.cross_entropy(output, target, reduction = 'sum').item()

      #맞힌 개수 계산
      pred = output.max(1, keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()
    
  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accuracy

for epoch in range(1, EPOCHS + 1):
  train(model, train_loader, optimizer)
  test_loss, test_accuracy = evaluate(model, test_loader)

  print('[{}] Test Loss: {:.4}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))


[1] Test Loss: 0.6307, Accuracy: 77.51%
[2] Test Loss: 0.5302, Accuracy: 80.96%
[3] Test Loss: 0.4884, Accuracy: 82.49%
[4] Test Loss: 0.4569, Accuracy: 83.71%
[5] Test Loss: 0.438, Accuracy: 84.31%
[6] Test Loss: 0.429, Accuracy: 84.64%
[7] Test Loss: 0.422, Accuracy: 84.69%
[8] Test Loss: 0.4142, Accuracy: 84.84%
[9] Test Loss: 0.3961, Accuracy: 85.72%
[10] Test Loss: 0.3902, Accuracy: 85.89%
[11] Test Loss: 0.384, Accuracy: 86.08%
[12] Test Loss: 0.379, Accuracy: 86.26%
[13] Test Loss: 0.379, Accuracy: 86.26%
[14] Test Loss: 0.3708, Accuracy: 86.69%
[15] Test Loss: 0.3683, Accuracy: 86.63%
[16] Test Loss: 0.3713, Accuracy: 86.46%
[17] Test Loss: 0.3569, Accuracy: 86.73%
[18] Test Loss: 0.3529, Accuracy: 86.96%
[19] Test Loss: 0.3486, Accuracy: 87.31%
[20] Test Loss: 0.3517, Accuracy: 87.26%
[21] Test Loss: 0.3476, Accuracy: 87.30%
[22] Test Loss: 0.3399, Accuracy: 87.72%
[23] Test Loss: 0.3442, Accuracy: 87.50%
[24] Test Loss: 0.3394, Accuracy: 87.81%
[25] Test Loss: 0.3393, Accurac